In [ ]:
!pip3 install slackweb

In [ ]:
# ライブラリインポート
import csv
import os
import sys
import requests
import slackweb
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request
import pprint
import slackweb

In [ ]:
# 取得したい情報をスクレイピングする
headers_dic = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"}
url = 'サイトのURL'
r = requests.get(url, headers=headers_dic)
soup = BeautifulSoup(r.text, 'html.parser')

# 取得したエピソードの題名をリストに格納する
result = []
for episode in soup.find_all(class_='取得したい箇所のクラス名'):
    result.append([
        episode.text
    ])
print(result)

In [ ]:
# googleドライブにアクセスする
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# googleドライブにあらかじめ空のcsvファイルを作成しておく
# csvファイルにアクセスする
with open('/content/drive/My Drive/Colab Notebooks/last_log.csv') as f:
    print(f.read())

In [ ]:
if not os.path.exists('/content/drive/My Drive/Colab Notebooks/last_log.csv'):
    raise Exception('ファイルがありません。')
if os.path.getsize('/content/drive/My Drive/Colab Notebooks/last_log.csv') == 0:
    raise Exception('ファイルの中身が空です。')

# アクセスしたcsvファイルの内容をリストに格納する
csv_list = pd.read_csv('/content/drive/My Drive/Colab Notebooks/last_log.csv').values.tolist()
print(csv_list)

# エピソードの更新を待たずにコードを確かめたいので、故意にリストの末尾を削除して差分を作る
csv_list.pop(-1)
print(csv_list)

In [ ]:
# csvファイルに取得したエピソードの情報を書き込む
with open('/content/drive/My Drive/Colab Notebooks/last_log.csv', 'w', newline='',encoding='utf_8') as file:
    headers = ['Title']
    writer = csv.writer(file)
    writer.writerow(headers)
    for row in result:
        writer.writerow(row)

In [ ]:
# 取得したエピソードの情報と1つ前に保存したcsvとの中身を比較して差分だけ新たなリストに格納する
return_list = []
for tmp in result:
    if tmp not in csv_list:
        return_list.append(tmp)
print(return_list)

# リストの内容を全て文字列に変換する
# slackに通知する時に文字列に変換しないとエラーが出るため
diff = ''.join(map(str, return_list))
print(diff)

In [ ]:
# あらかじめslackのWebhook URLを取得しておく
# 前のコードで比較した差分があればslackに通知する
if return_list:
    text = '<!channel>\nSPY×FAMILYのエピソードが追加されました\n' #@channelメンション
    text += diff+ '\n'
    slack  = slackweb.Slack(url='コピーしたWebhook URL')
    slack.notify(text=text)